# Example: Compare MIPS of RISC-V Instruction Set Simulators

TODO

## Supported components

**Models:** Any (`sine_model` used below)

**Frontends:** Any (`tflite` used below)

**Frameworks/Backends:** Any (`tvmaotplus` used below)

**Platforms/Targets:** `etiss_pulpino`, `spike`, `ovpsim` (all of them used below)

## Prerequisites

Set up MLonmCU as usual, i.e. initializa an environment and install all required dependencies. Feel free to use the following minimal `environment.yml.j2` template:

```yaml
---
TODO
```

Do not forget to set your `MLONMCU_HOME` environment variable first if not using the default location!

## Usage

If supported by the defined target, the measured MIPS (of the Simulation) is part of the report printed/returned my MLonMCU. The following shows you how to get rid of unwanted further information and how to increase the accuracy of the MIPS value.

### A) Command Line Interface

Let's start with an example benchmark of two models using 2 different RISC-V simulators: (TODO: add QEMU)

In [22]:
!mlonmcu flow run resnet toycar --backend tvmaot --target etiss_pulpino --target spike

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - Loading extensions.py (User)
INFO - [session-262]  Processing stage LOAD
INFO - [session-262]  Processing stage BUILD
INFO - [session-262]  Processing stage COMPILE
INFO - [session-262]  Processing stage RUN
INFO - All runs completed successfuly!
INFO - Postprocessing session report
INFO - [session-262] Done processing runs
INFO - Report:
   Session  Run   Model Frontend Framework Backend Platform         Target    Cycles  MIPS  Total ROM  Total RAM  ROM read-only  ROM code  ROM misc  RAM data  RAM zero-init data Features                                             Config Postprocesses Comment
0      262    0  resnet   tflite       tvm  tvmaot     mlif  etiss_pulpino  82445446  69.0     233324     124785         167488     65692       144      2485              122300       []  {'tflite.use_inout_data': False, 'tflite.visua...            []       -
1      262    1  resnet   tflite       tvm  tvmaot 

The MIPS value can be found in the column next to the Cycles (which are in this case actually counting instructions). However there is a lot of further information we want to filter out next. This can be achieved using the `filter_cols` subprocess.

In [23]:
!mlonmcu flow run resnet toycar --backend tvmaot --target etiss_pulpino --target spike --postprocess filter_cols --config filter_cols.keep="Model,Target,MIPS"

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - Loading extensions.py (User)
INFO - [session-263]  Processing stage LOAD
INFO - [session-263]  Processing stage BUILD
INFO - [session-263]  Processing stage COMPILE
INFO - [session-263]  Processing stage RUN
INFO - [session-263]  Processing stage POSTPROCESS
INFO - All runs completed successfuly!
INFO - Postprocessing session report
value 'Model,Target,MIPS'
value 'Model,Target,MIPS'
value 'Model,Target,MIPS'
INFO - [session-263] Done processing runs
INFO - Report:
    Model         Target  MIPS
0  resnet  etiss_pulpino  68.0
1  resnet          spike   NaN
2  toycar  etiss_pulpino   2.0
3  toycar          spike   NaN


That looks much more clean! However the numbers seem quite low, especially for the smaller `toycar` (MLPerfTiny Anomaly Detection) model. Let's see if the MIPS will increase when running more than a single inference. We are using the `benchmark` feature for this.

*Hint*: Since we are now running our benchmarks 60 times more often, the following cell will likely need a few minutes to execute.

In [25]:
!mlonmcu flow run resnet toycar --backend tvmaot --target etiss_pulpino --target spike --postprocess config2cols --postprocess filter_cols --config filter_cols.keep="Model,Target,MIPS,config_benchmark.num_runs" --feature benchmark --config-gen benchmark.num_runs=1 --config-gen benchmark.num_runs=10 --config-gen benchmark.num_runs=50

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - Loading extensions.py (User)
INFO - [session-265]  Processing stage LOAD
INFO - [session-265]  Processing stage BUILD
INFO - [session-265]  Processing stage COMPILE
INFO - [session-265]  Processing stage RUN
INFO - [session-265]  Processing stage POSTPROCESS
INFO - All runs completed successfuly!
INFO - Postprocessing session report
value 'Model,Target,MIPS,config_benchmark.num_runs'
value 'Model,Target,MIPS,config_benchmark.num_runs'
value 'Model,Target,MIPS,config_benchmark.num_runs'
INFO - [session-265] Done processing runs
INFO - Report:
     Model         Target   MIPS config_benchmark.num_runs
0   resnet  etiss_pulpino   68.0                         1
1   resnet          spike    NaN                         1
2   resnet  etiss_pulpino  130.0                        10
3   resnet          spike    NaN                        10
4   resnet  etiss_pulpino  137.0                        50
5   resnet

This look more promising. This experiment shows MIPS measurements might not be accurate for short-running simulations.

### B) Python Scripting

TODO

Use pandas instead of postprocess

In [3]:
from tempfile import TemporaryDirectory
from pathlib import Path
import pandas as pd

from mlonmcu.context.context import MlonMcuContext
from mlonmcu.session.run import RunStage

Benchmark Configuration

In [4]:
FRONTEND = "tflite"
MODEL = "sine_model"
BACKEND = "tvmaotplus"
PLATFORM = "mlif"
TARGET = "etiss_pulpino"
FEATURES = ["log_instrs"]
CONFIG = {"log_instrs.to_file": True}
POSTPROCESSES = ["analyse_instructions"]

Initialize and run a single benchmark

In [5]:
with MlonMcuContext() as context:
    session = context.create_session()
    run = session.create_run(config=CONFIG)
    run.add_features_by_name(FEATURES, context=context)
    run.add_frontend_by_name(FRONTEND, context=context)
    run.add_model_by_name(MODEL, context=context)
    run.add_backend_by_name(BACKEND, context=context)
    run.add_platform_by_name(PLATFORM, context=context)
    run.add_target_by_name(TARGET, context=context)
    run.add_postprocesses_by_name(POSTPROCESSES)
    session.process_runs(context=context)
    report = session.get_reports()
report.df

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - Loading extensions.py (User)
INFO - [session-241] Processing all stages
WARNING - unexpected script output (cycles)
ERROR - 'builtin_function_or_method' object is not subscriptable
Traceback (most recent call last):
  File "/var/tmp/ga87puy/mlonmcu/mlonmcu/venv/lib/python3.8/site-packages/mlonmcu-0.3.0.dev0-py3.8.egg/mlonmcu/session/run.py", line 782, in process
    func()
  File "/var/tmp/ga87puy/mlonmcu/mlonmcu/venv/lib/python3.8/site-packages/mlonmcu-0.3.0.dev0-py3.8.egg/mlonmcu/session/run.py", line 549, in postprocess
    artifacts = postprocess.post_run(temp_report, self.artifacts)
  File "/var/tmp/ga87puy/mlonmcu/mlonmcu/venv/lib/python3.8/site-packages/mlonmcu-0.3.0.dev0-py3.8.egg/mlonmcu/session/run.py", line 507, in artifacts
    itertools.chain([subs[subs.keys[0]] for stage, subs in self.artifacts_per_stage.items()])
  File "/var/tmp/ga87puy/mlonmcu/mlonmcu/venv/lib/python3.8/site-package

,Session,Run,Model,Frontend,Framework,Backend,Platform,Target,Total ROM,Total RAM,ROM read-only,ROM code,ROM misc,RAM data,RAM zero-init data,Features,Config,Postprocesses,Comment,Failing
0,241,0,sine_model,tflite,tvm,tvmaotplus,mlif,etiss_pulpino,56100,2737,4280,51676,144,2493,244,[log_instrs],"{'tflite.use_inout_data': False, 'tflite.visua...",[analyse_instructions],-,True
